In [15]:
import random
import numpy as np
from matplotlib import pyplot as plt
import cv2
import copy

lab_boundaries = [([190, 120, 138], [240, 138, 177])]
[(lower,upper)] = lab_boundaries
lower = np.array(lower, dtype="uint8")
upper = np.array(upper, dtype="uint8")

fourcc = cv2.VideoWriter_fourcc(*'XVID')

cap = cv2.VideoCapture('day2_lane1/0.jpg')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('test.avi', fourcc, 30.0,(frame_width,frame_height))
out2 = cv2.VideoWriter('fittest3.avi', fourcc, 30.0,(frame_width,frame_height))

bounds = [300, 600]

In [11]:
def find_first(item, vec):
    """return the index of the first occurence of item in vec"""
    for i in range(len(vec)):
        if item == vec[i]:
            return i
    return -1

In [177]:
fname = 'day2_lane1/120.jpg'
blank=np.zeros((frame_height,frame_width,3),np.uint8)
original = cv2.imread(fname, cv2.IMREAD_COLOR)
filtered = copy.deepcopy(original)
# filtered = cv2.bilateralFilter(filtered,9,75,75)
        
lab_image = cv2.cvtColor(filtered, cv2.COLOR_BGR2LAB)

mask = cv2.inRange(lab_image, lower, upper)
output = cv2.bitwise_and(lab_image, lab_image, mask=mask)
        
canny = copy.deepcopy(output)     
canny = cv2.Canny(canny,100,200)     
              
window = canny[bounds[1]:bounds[0]:-1].transpose()
xy = []
for k in range(len(window)):
    col = window[k]
    j = find_first(255, col)
    if j != -1:
        xy.extend((k, j))
# Reshape into [[x1, y1],...]
data = np.array(xy).reshape((-1, 2))
# Translate points back to original positions.
data[:, 1] = bounds[1] - data[:, 1]
pts = data.reshape((-1, 1, 2)) 
        
xdata = data[:,0]
ydata = data[:,1]
        
z = np.polyfit(xdata,ydata,3)
oneD = np.polyfit(xdata,ydata,1)
oned = np.poly1d(oneD)
onedcoef = oned.c[0]
orthoned = -1/onedcoef
        
f = np.poly1d(z)
        
fitxy = []
for n in range(bounds[1]):
    xfit = n
    yfit = int(f(n))
    fitxy.append([xfit,yfit])
fitxy = np.array(fitxy).reshape((-1,2))

fitpts = fitxy.reshape((-1,1,2))
        
output2 = cv2.polylines(original, [fitpts], isClosed=False, color=(255,0,0))

In [178]:
output3 = cv2.polylines(blank, [fitpts], isClosed=False, color=(255,0,0))
right=onedcoef
left=right*orthoned*260
rows,cols,depth = original.shape

M = np.float32([[1,0,right],[0,1,left]])
output4 = cv2.warpAffine(output3,M,(cols,rows))

canny4 = cv2.Canny(output4,100,200)
              
window2 = canny4[bounds[1]:0:-1].transpose()
xy2 = []
for k in range(len(window2)):
    col = window2[k]
    j = find_first(255, col)
    if j != -1:
        xy2.extend((k, j))
# Reshape into [[x1, y1],...]
data2 = np.array(xy2).reshape((-1, 2))
# Translate points back to original positions.

data2[:, 1] = bounds[1] - data2[:, 1]
pts2 = data2.reshape((-1, 1, 2)) 
        
xdata2 = data2[:,0]
ydata2 = data2[:,1]
        
z2 = np.polyfit(xdata2,ydata2,3)
        
f2 = np.poly1d(z2)
        
fitxy2 = []
for n in range(bounds[1]):
    xfit = n
    yfit = int(f2(n))
    fitxy2.append([xfit,yfit])
fitxy2 = np.array(fitxy2).reshape((-1,2))

fitpts2 = fitxy2.reshape((-1,1,2))
output5= cv2.polylines(output2, [fitpts2], isClosed=False, color=(255,255,0))
cv2.imshow('out',output5)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [173]:
original.shape

(600, 600, 3)